In [1]:
%env CUDA_VISIBLE_DEVICES=6

import os
import logging

os.environ.pop("HF_HUB_OFFLINE", None)
logging.getLogger().setLevel(logging.ERROR)  # or logging.CRITICAL

import torch
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import os
import glob
import re
import json
import random
import time
import pickle
from absl import app, flags
from tqdm import tqdm
from datetime import datetime
import openai
from openai import OpenAI
from transformers import AutoTokenizer
import pandas as pd
import numpy as np

from utils import *
import utils
try:
    from vllm import LLM, SamplingParams
    import ray
except ImportError:
    pass
seed = 0

env: CUDA_VISIBLE_DEVICES=6
INFO 05-06 02:39:10 [__init__.py:239] Automatically detected platform cuda.


In [2]:
import subprocess
import torch
def get_freest_cuda_device():
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=memory.free', '--format=csv,nounits,noheader'],
        stdout=subprocess.PIPE, encoding='utf-8')
    memory_free = [int(x) for x in result.stdout.strip().split('\n')]
    return memory_free.index(max(memory_free))

best_gpu = get_freest_cuda_device()
device = torch.device(f"cuda:{best_gpu}")
print(f"Using GPU: {device}")
# %env CUDA_VISIBLE_DEVICES=0

Using GPU: cuda:2


In [2]:
with open(os.path.abspath('../ryan_openai.txt'), 'r') as f:
    utils.client = OpenAI(api_key=f.read().rstrip('\n'))

In [3]:
import os

# Explicitly unset all offline-related env vars
os.environ.pop("HF_HUB_OFFLINE", None)
os.environ.pop("TRANSFORMERS_OFFLINE", None)
os.environ["HF_HUB_OFFLINE"] = "0"
os.environ["TRANSFORMERS_OFFLINE"] = "0"

with open("../token.txt", "r") as f:
    token = f.read().strip()

from huggingface_hub import login
login(token=token)

FileNotFoundError: [Errno 2] No such file or directory: '../token.txt'

In [4]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("Tianyi-Lab/Personas")
personas_chatting_array = ds['train']['Llama-3.1-70B-Instruct_descriptive_persona']

In [5]:
np.random.seed(seed)
random_indices = np.random.choice(len(personas_chatting_array), size=200, replace=False).astype(int)
personas_chatting_array = np.array(personas_chatting_array)[random_indices]

In [6]:
import json
import shutil
import os
config_chatting = {'agent1_role': 'Person A',
                  'agent2_role': 'Person B',
                  'agent1_prompt': "You are %SPEAKER_ROLE%, and you are having an online conversation with %LISTENER_ROLE%. Each of you is chatting to get to know each other, taking turns asking questions, and sharing stories about your lives, careers, and experiences. The goal is to try to find something in common. Do not only ask questions, you should also share details about yourself. This is a brief story that you have written about yourself: %SPEAKER_BACKSTORY%",
                  'agent2_prompt': "You are %SPEAKER_ROLE%, and you are having an online conversation with %LISTENER_ROLE%. Each of you is chatting to get to know each other, taking turns asking questions, and sharing stories about your lives, careers, and experiences. The goal is to try to find something in common. Do not only ask questions, you should also share details about yourself. This is a brief story that you have written about yourself: %SPEAKER_BACKSTORY%",
                  'reminder_prompt': "Keep your response very brief — 2 sentences or less. Do NOT repeat anything you've already said: \n",
                  "scenario": "Two people are getting to know one another through an online conversation. ",
                  'eval_prompts': {
                  'prompt_consistency': """You are evaluating whether the following utterance by %SPEAKER_ROLE% is consistent with their given background and the task instructions. Answer YES if the line directly contradicts any part of the background or task descriptio—this includes contradictions in facts, behavior, beliefs, emotional stance, or conversational strategy. Answer NO if the line introduces new details, but those details are **plausibly aligned** with the background. Be strict and literal in your evaluation: if the line violates the persona’s stated traits, mark it as a contradiction even if the violation is subtle. Background for %SPEAKER_ROLE%: %SPEAKER_BACKSTORY% Line spoken by %SPEAKER_ROLE%: %SPEAKER_LINE%  Answer YES if the line contradicts the background, and NO if it does not. Then, explain your answer in one sentence. Be precise and avoid vague justification."""}}

# os.makedirs("chatting", exist_ok=True)
# with open("chatting/config_chatting.json", "w", encoding="utf-8") as f:
#     json.dump(config_chatting, f, indent=4)

In [7]:
llms = ["Llama-3.1-8B-Instruct", "gpt-4o-mini", "Qwen2.5-3B-Instruct", "Llama-3.1-8B", "Mistral-7B-Instruct", "Llama-3.1-70B", "Llama-3.1-70B-Instruct", "phi-3.5-mini-instruct"]
        
config_llm = {'agent1_model': '/raid/users/ryan_cheng/checkpoints/Chatting/llama3-8b-kto-sft-prompt',
             'agent2_model': 'Llama-3.1-8B-Instruct',
             'eval_model': 'Llama-3.1-8B-Instruct',
             'iterations': 10,
             'verbose': False,
             'write': True,
             'convo_length_limit': 10,
             'max_tokens': 256,
             'gpus': 1,
             'seed': 0,
             'task_name': 'Chatting',
             'model_dir': "/raid/users/ryan_cheng/models/"}

# with open("chatting/Llama-3.1-8B-Instruct.json", "w", encoding="utf-8") as f:
#     json.dump(config_llm, f, indent=4)

In [8]:
# def get_text_after_colon(label, text):
#     """
#     Extracts the text that appears after a given label followed by a colon.
#     Example: get_text_after_colon("Name", "Name: John Doe") returns "John Doe"
#     """
#     pattern = rf"{re.escape(label)}:\s*(.*)"
#     match = re.search(pattern, text)
#     return match.group(1).strip() if match else text

# personas_chatting = []

# for persona in personas_chatting_array:
#     name_prompt = "Given the following biography below, parse the name of the person. \nOnly output the name.\n" + persona + "\nName: "
#     name = completion_create("gpt-4o-mini", config_llm, name_prompt)
#     name = get_text_after_colon("Name", name)
#     personas_chatting.append({'persona': persona, 'name': name})

In [9]:
# with open('chatting/config_chatting_personas.json', 'w') as f:
#     json.dump(personas_chatting, f, indent=4)

In [10]:
with open("chatting/config_chatting_personas.json", "r") as f:
    personas_chatting = json.load(f)

In [11]:
import re

def get_first_name(full_name):
    return full_name.strip().split()[0]
    
def clean_role_prefix(response, expected_role):
    """
    Removes repeated instances of the expected_role prefix at the start (e.g., 'Therapist: Therapist:'),
    and ensures the response begins with a single correct expected_role prefix.
    """
    pattern = rf"^(({re.escape(expected_role)}):\s*)+"
    cleaned = re.sub(pattern, '', response.strip(), flags=re.IGNORECASE)
    return cleaned
    
def is_role_confused(response, other_role):
    """
    Checks if the output starts with the wrong speaker tag.
    """
    if other_role + ":" in response:
        return True
    else: 
        return False

def generate_response(agent_model, expected_role, other_role, config_llm, prompt, max_retries=3):
    for _ in range(max_retries):
        response = completion_create(agent_model, config_llm, prompt)
        print(expected_role)
        if not is_role_confused(response, other_role):
            return clean_role_prefix(response, expected_role)
            
    return clean_role_prefix(response, expected_role)

def generate_chat(config_llm, p1, p2, p1_name, p2_name, pturn=1):
    stats['P1'] = p1
    stats['P2'] = p2
    stats['pturn'] = pturn
    config_chatting["agent1_role"] = get_first_name(p1_name)
    config_chatting["agent2_role"] = get_first_name(p2_name)

    round_num = 0
    while round_num < config_llm['convo_length_limit']:
        conversation = ("".join([turn[1] if isinstance(turn, tuple) else turn for turn in stats["conversation"]]) if len(stats["conversation"]) != 0 else "You are starting the conversation.\n")
        
        if pturn == 1:
            prompt = config_chatting["agent1_prompt"]
            pturn = 2
            if config_llm["verbose"]:
                print(prompt)
                print()

            if round_num!=0: 
                prompt+= "Your conversation so far is below:\nConversation: %CONVERSATION%"
                
            if round_num >=config_llm['convo_length_limit']*2-11 and round_num<=config_llm['convo_length_limit']*2-1:
                prompt+= "You have " + str((config_llm['convo_length_limit']-round_num)//2) + " rounds left." + "Make sure to conclude the conversation as you're near the end."

            elif round_num>config_llm['convo_length_limit']*2-1:
                prompt+= "This is your concluding line in the conversation."

            if round_num!=0: 
                prompt+= "Continue the conversation with " + config_chatting["agent2_role"] +  ". Remember you are " +  config_chatting["agent1_role"] + "."
                 
            prompt += config_chatting["reminder_prompt"] + "DO NOT PREFACE THE RESPONSE WITH THIRD-PERSON STATEMENTS SUCH AS \"Sure, here's a response from...\"\n"
            prompt+="%SPEAKER_ROLE%:"
            prompt = prompt.replace("%SPEAKER_ROLE%", config_chatting["agent1_role"]) \
                           .replace("%LISTENER_ROLE%", config_chatting["agent2_role"]) \
                           .replace("%SPEAKER_BACKSTORY%", p1) \
                           .replace("%CONVERSATION%", conversation)
            
            response = generate_response(config_llm['agent1_model'], config_chatting["agent1_role"], config_chatting["agent2_role"], config_llm, prompt)

            while "\n\n" in response:
                print(response)
                response = generate_response(config_llm['agent1_model'], config_chatting["agent1_role"], config_chatting["agent2_role"], config_llm, prompt)
            
            if config_chatting["agent2_role"] + ":" in response:
                response = response[:response.index(config_chatting["agent2_role"] + ":")]
            stats["conversation"].append((round_num, f"{config_chatting["agent1_role"]}: " + response + "\n"))
        
        else:
            prompt = config_chatting["agent2_prompt"]
            pturn = 1    
            if config_llm["verbose"]:
                print(prompt)
                print()

            if round_num!=0: 
                prompt+= "Your conversation so far is below:\nConversation: %CONVERSATION%"
            if round_num >=config_llm['convo_length_limit']*2-11 and round_num<=config_llm['convo_length_limit']*2-1:
                prompt+= "You have " + str((config_llm['convo_length_limit']-round_num)//2) + " rounds left." + "Make sure to conclude the conversation as you're near the end."
            elif round_num>config_llm['convo_length_limit']*2-1:
                prompt+= "This is your concluding line in the conversation."

            if round_num!=0: 
                prompt+= "Continue the conversation with " + config_chatting["agent1_role"] +  ". Remember you are " +  config_chatting["agent2_role"] + "."

            prompt += config_chatting["reminder_prompt"] + "DO NOT PREFACE THE RESPONSE WITH THIRD-PERSON STATEMENTS SUCH AS \"Sure, here's a response from...\"\n"
            prompt+="%SPEAKER_ROLE%:"
            prompt = prompt.replace("%SPEAKER_ROLE%", config_chatting["agent2_role"]) \
                           .replace("%LISTENER_ROLE%", config_chatting["agent1_role"]) \
                           .replace("%SPEAKER_BACKSTORY", p2) \
                           .replace("%CONVERSATION%", conversation)

            response = generate_response(config_llm['agent2_model'], config_chatting["agent2_role"], config_chatting["agent1_role"], config_llm, prompt)
            
            while "\n\n" in response:
                print(response)
                response = generate_response(config_llm['agent2_model'], config_chatting["agent2_role"], config_chatting["agent1_role"], config_llm, prompt)
            
            if config_chatting["agent1_role"] + ":" in response:
                response = response[:response.index(config_chatting["agent1_role"] + ":")]

            stats["conversation"].append((round_num, f"{config_chatting["agent2_role"]}: " + response + "\n"))
        round_num += 1

    stats["rounds"] = round_num
    if config_llm['verbose']:
        print(stats["conversation"])
    return stats.copy()

def reset_stats():
    stats_template = {
        "task_name": config_llm['task_name'],
        "P1": "",
        "P2": "",
        "conversation": [],
        "pturn": 0, # beginning person (1 or 2)
        "index": -1,
        "timestamp": "",
        "rounds": 0,
        'conversation_only': True
    }
    for key, value in stats_template.items():
        stats[key] = value

In [12]:
import os
import random
from datetime import datetime
import utils
utils.config = config_llm

current_date = str(datetime.now().strftime("%m.%d.%y"))
output_dir = f"chatting/exp/{current_date}"
os.makedirs(output_dir, exist_ok=True)

# Generate unique random number for filename
def generate_unique_file_number(output_dir, prefix, seed, extension=".json"):
    while True:
        rand_num = random.randint(0, 1000)
        filename = f"{prefix}_{seed}_{rand_num}{extension}"
        filepath = os.path.join(output_dir, filename)
        if not os.path.exists(filepath):
            return rand_num

unique_num = generate_unique_file_number(
    output_dir,
    config_llm['agent1_model'],
    config_llm['seed']
)

# File to write output to
write_file = os.path.join(output_dir, f"kto_{config_llm['agent1_model'][config_llm['agent1_model'].rfind("/")+1:]}_{config_llm['seed']}_{unique_num}.json")

In [ ]:
lengths = [10, 20, 40, 60]
write_files = [os.path.join(output_dir, f"{config_llm['agent1_model'][config_llm['agent1_model'].rfind("/")+1:]}_{config_llm['seed']}_{length}_{unique_num}.json") for length in lengths]

In [11]:
len(personas_chatting)

200

In [12]:
test_prompt = "You are Alexandra, and you are having an online conversation with Ethan. Each of you is chatting to get to know each other, taking turns asking questions, and sharing stories about your lives, careers, and experiences. The goal is to try to find something in common. Do not only ask questions, you should also share details about yourself. This is a brief story that you have written about yourself: \"Meet 49-year-old Alexandra \\\"Alex\\\" Thompson, a vibrant and determined woman living in Charleston, South Carolina. Born and raised in the heart of the Lowcountry, Alex embodies the warmth and hospitality that her home state is famous for. As a white woman, she has had her share of privileges, but she's also faced challenges that have shaped her into the strong, independent individual she is today.\\n\\nAlex grew up in a traditional Southern family, where her parents valued hard work, education, and community involvement. Her father was a high school history teacher, and her mother worked as a librarian, instilling in Alex a love for learning and a strong sense of social responsibility. She has two younger brothers, both of whom have moved away from South Carolina to pursue their careers.\\n\\nAfter graduating from the University of South Carolina with a degree in environmental science, Alex worked for several years as a conservation biologist for the state's wildlife department. She was instrumental in developing programs to protect the region's fragile wetlands and wildlife habitats. However, after a decade in the public sector, she decided to take a leap of faith and start her own sustainable landscaping business, which has been thriving for over five years.\\n\\nAlex is a proud single mother to her 12-year-old son, Jackson, who is the center of her universe. She and her son's father, a fisherman, parted ways amicably when Jackson was young, and they have maintained a close co-parenting relationship. Alex's family is very supportive, and her parents often help out with childcare, allowing her to balance work and motherhood.\\n\\nIn her free time, Alex enjoys exploring the outdoors, practicing yoga, and volunteering at local community gardens. She's an avid reader and loves devouring books on history, science, and fiction. Her favorite authors include Pat Conroy and Jesmyn Ward, both of whom have a deep connection to the South.\\n\\nAlex's home is a cozy, eco-friendly bungalow in the heart of Charleston, filled with plants, artwork, and mementos from her travels. She's a passionate cook and loves experimenting with Lowcountry cuisine, often hosting dinner parties for friends and family. Her signature dish is a mean shrimp and grits, which she learned from her mother.\\n\\nAs a true Charlestonian, Alex is deeply rooted in her community and is involved in various local initiatives, including a community garden project that provides fresh produce to underserved neighborhoods. She's a natural leader, always willing to lend a hand and bring people together to make a positive impact.\\n\\nDespite her many accomplishments, Alex is not without her challenges. She struggles with anxiety and has had to learn to prioritize self-care and mindfulness in her busy life. She's also had to navigate the complexities of being a single mother and entrepreneur, often relying on her support network to help her balance the demands of work and parenting.\\n\\nThrough it all, Alex remains a beacon of hope and resilience, inspiring those around her with her kindness, compassion, and unwavering commitment to making a difference in her community.\"Keep your response very brief \u2014 2 sentences or less. Do NOT repeat anything you've already said: \nDO NOT PREFACE THE RESPONSE WITH THIRD-PERSON STATEMENTS SUCH AS \"Sure, here's a response from...\"\nAlexandra:"


In [13]:
completion_create(config_llm['agent1_model'], config_llm, test_prompt)

using model  /raid/users/ryan_cheng/checkpoints/Chatting/llama-8b-sft-ppo-prompt
Attempting to use tokenizer for /raid/users/ryan_cheng/checkpoints/Chatting/llama-8b-sft-ppo-prompt
Info: using finetuned model setup
[2025-05-06 02:21:36,022] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/nfs/kun2/users/ryan_cheng/miniconda3/envs/openrlhf/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/nfs/kun2/users/ryan_cheng/miniconda3/envs/openrlhf/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /nfs/kun2/users/ryan_cheng/miniconda3/envs/openrlhf/lib/libcufile.so: undefined reference to `dlvsym'
/nfs/kun2/users/ryan_cheng/miniconda3/envs/openrlhf/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /nfs/kun2/users/ryan_cheng/miniconda3/envs/openrlhf/lib/libcufile.so: undefined reference to `dlopen'
/nfs/kun2/users/ryan_cheng/miniconda3/envs/openrlhf/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /nfs/kun2/users/ryan_cheng/miniconda3/envs/openrlhf/lib/libcufile.so: undefined reference to `dlclose'
/nfs/kun2/users/ryan_cheng/miniconda3/envs/op

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


" Hi Ethan! I'm excited to be chatting with you. I'm a big fan of history, especially the Civil War era, and I've always been fascinated by the stories of the people who lived through it. What about you, what are some of your interests?"

In [14]:
config_llm

{'agent1_model': '/raid/users/ryan_cheng/checkpoints/Chatting/llama-8b-sft-ppo-prompt',
 'agent2_model': 'Llama-3.1-8B-Instruct',
 'eval_model': 'Llama-3.1-8B-Instruct',
 'iterations': 10,
 'verbose': False,
 'write': True,
 'convo_length_limit': 10,
 'max_tokens': 256,
 'gpus': 1,
 'seed': 0,
 'task_name': 'Chatting',
 'model_dir': '/raid/users/ryan_cheng/models/'}

In [15]:
utils.config['fp8'] = True

In [16]:
utils.config['fp8']

True

In [17]:
%%capture
# index_offset = load_stats_file(write_file)
conversations = []    

for i in range(1):
    for p_dict1, p_dict2 in tqdm(np.array(personas_chatting)[:20].reshape(-1, 2)):
        for j, convo_length in enumerate(lengths):
            index_offset = load_stats_file(write_files[j])
            config_llm['convo_length_limit'] = convo_length
            reset_stats()
            conversation = generate_chat(
                config_llm,
                p_dict1["persona"], 
                p_dict2["persona"],
                p_dict1["name"], 
                p_dict2["name"], 
                pturn=1
            )
            print(conversation)
            conversations.append(conversation)
            stats['index'] = index_offset
            stats['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            write_stats(write_files[j])
            index_offset += 1


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


INFO 05-06 02:22:12 [config.py:717] This model supports multiple tasks: {'generate', 'embed', 'reward', 'score', 'classify'}. Defaulting to 'generate'.
INFO 05-06 02:22:12 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=16384.
WARNING 05-06 02:22:14 [utils.py:2382] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/getting_started/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
INFO 05-06 02:22:18 [__init__.py:239] Automatically detected platform cuda.
INFO 05-06 02:22:21 [core.py:58] Initializing a V1 LLM engine (v0.8.5) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=

[W506 02:22:26.274958757 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W506 02:22:26.275421425 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3


INFO 05-06 02:22:26 [parallel_state.py:1004] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0
INFO 05-06 02:22:26 [cuda.py:221] Using Flash Attention backend on V1 engine.
WARNING 05-06 02:22:26 [topk_topp_sampler.py:69] FlashInfer is not available. Falling back to the PyTorch-native implementation of top-p & top-k sampling. For the best performance, please install FlashInfer.
INFO 05-06 02:22:26 [gpu_model_runner.py:1329] Starting to load model meta-llama/Meta-Llama-3.1-8B-Instruct...
INFO 05-06 02:22:28 [weight_utils.py:265] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:01<00:05,  1.88s/it]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:03<00:03,  1.98s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:04<00:01,  1.38s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:06<00:00,  1.63s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:06<00:00,  1.65s/it]



INFO 05-06 02:22:35 [loader.py:458] Loading weights took 6.78 seconds
INFO 05-06 02:22:35 [gpu_model_runner.py:1347] Model loading took 14.9889 GiB and 8.299951 seconds
INFO 05-06 02:23:07 [backends.py:420] Using cache directory: /home/ryan_cheng/.cache/vllm/torch_compile_cache/8cd4dbc093/rank_0_0 for vLLM's torch.compile
INFO 05-06 02:23:07 [backends.py:430] Dynamo bytecode transform time: 32.16 s
INFO 05-06 02:23:11 [backends.py:118] Directly load the compiled graph(s) for shape None from the cache, took 4.005 s
INFO 05-06 02:23:12 [monitor.py:33] torch.compile takes 32.16 s in total
INFO 05-06 02:23:13 [kv_cache_utils.py:634] GPU KV cache size: 273,936 tokens
INFO 05-06 02:23:13 [kv_cache_utils.py:637] Maximum concurrency for 131,072 tokens per request: 2.09x
INFO 05-06 02:23:32 [gpu_model_runner.py:1686] Graph capturing finished in 19 secs, took 0.66 GiB
INFO 05-06 02:23:32 [core.py:159] init engine (profile, create kv cache, warmup model) took 57.29 seconds
INFO 05-06 02:23:32 [co

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [ ]:
def eval_prompt_consistency(conv_dict):
    #assert 'eval_prompt_consistency' not in conv_dict # warn if we are replacing metrics we don't mean to overwrite
    conv_dict['eval_prompt_consistency'] = []
    conv_dict['P1_prompt_consistency_score'] = 0
    conv_dict['P2_prompt_consistency_score'] = 0
    p2_utterances = 0
    p1_utterances = 0
    pturn = conv_dict["pturn"]
    for line in conv_dict["conversation"]:
        line_number = line[0]
        convo_line = line[1]
        if pturn == 1:
            prompt = config_chatting["eval_prompts"]["prompt_consistency"].replace("%SPEAKER_ROLE%", config_chatting["agent1_role"]) \
                                                                          .replace("%SPEAKER_BACKSTORY%", conv_dict["P1"]) \
                                                                          .replace("%SPEAKER_LINE%", convo_line)
            if config_llm['verbose']:
                print(prompt)
            output = completion_create(config_llm['eval_model'], config, prompt)
            conv_dict['eval_prompt_consistency'].append((line_number, output))
            if "YES" not in output: # no contradiction
                conv_dict['P1_prompt_consistency_score'] += 1
            p1_utterances += 1
            pturn = 2
        elif pturn == 2:
            prompt = config_chatting["eval_prompts"]["prompt_consistency"].replace("%SPEAKER_ROLE%", config_chatting["agent2_role"]) \
                                                                          .replace("%SPEAKER_BACKSTORY%", conv_dict["P2"]) \
                                                                          .replace("%SPEAKER_LINE%", convo_line)
            if config_llm['verbose']:
                print(prompt)
            output = completion_create(config_llm['eval_model'], config, prompt)
            conv_dict['eval_prompt_consistency'].append((line_number, output))
            if "YES" not in output: # no contradiction
                conv_dict['P2_prompt_consistency_score'] += 1
            p2_utterances += 1
            pturn = 1 
            
    if p1_utterances > 0:
        conv_dict['P1_prompt_consistency_score'] /= p1_utterances

    if p2_utterances > 0:
        conv_dict['P2_prompt_consistency_score'] /= p2_utterances
        
    print(conv_dict)
    return conv_dict

In [ ]:
import utils
utils.config = config_llm

with open("chatting/exp/04.22.25/Llama-3.1-8B-Instruct_0_724.json", "r") as f:
    conversations = json.load(f)

for conversation in tqdm(conversations):
    conversation = eval_prompt_consistency(conversation)

write_stats(write_file)

In [ ]:
with open(write_file, "w", encoding="utf-8") as f:
    json.dump(conversations, f, indent=4)

In [ ]:
conversations